In [595]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import re

pd.set_option('display.max_columns',None)

In [596]:
df = pd.read_csv('./smartphones - smartphones.csv')

In [597]:
df.head()

,model,price,rating,sim,processor,ram,battery,display,camera,card,os
0,OnePlus 11 5G,"₹54,999",89.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC","Snapdragon 8 Gen2, Octa Core, 3.2 GHz Processor","12 GB RAM, 256 GB inbuilt",5000 mAh Battery with 100W Fast Charging,"6.7 inches, 1440 x 3216 px, 120 Hz Display wit...",50 MP + 48 MP + 32 MP Triple Rear & 16 MP Fron...,Memory Card Not Supported,Android v13
1,OnePlus Nord CE 2 Lite 5G,"₹19,989",81.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi","Snapdragon 695, Octa Core, 2.2 GHz Processor","6 GB RAM, 128 GB inbuilt",5000 mAh Battery with 33W Fast Charging,"6.59 inches, 1080 x 2412 px, 120 Hz Display wi...",64 MP + 2 MP + 2 MP Triple Rear & 16 MP Front ...,"Memory Card (Hybrid), upto 1 TB",Android v12
2,Samsung Galaxy A14 5G,"₹16,499",75.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi","Exynos 1330, Octa Core, 2.4 GHz Processor","4 GB RAM, 64 GB inbuilt",5000 mAh Battery with 15W Fast Charging,"6.6 inches, 1080 x 2408 px, 90 Hz Display with...",50 MP + 2 MP + 2 MP Triple Rear & 13 MP Front ...,"Memory Card Supported, upto 1 TB",Android v13
3,Motorola Moto G62 5G,"₹14,999",81.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi","Snapdragon 695, Octa Core, 2.2 GHz Processor","6 GB RAM, 128 GB inbuilt",5000 mAh Battery with Fast Charging,"6.55 inches, 1080 x 2400 px, 120 Hz Display wi...",50 MP + 8 MP + 2 MP Triple Rear & 16 MP Front ...,"Memory Card (Hybrid), upto 1 TB",Android v12
4,Realme 10 Pro Plus,"₹24,999",82.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi","Dimensity 1080, Octa Core, 2.6 GHz Processor","6 GB RAM, 128 GB inbuilt",5000 mAh Battery with 67W Fast Charging,"6.7 inches, 1080 x 2412 px, 120 Hz Display wit...",108 MP + 8 MP + 2 MP Triple Rear & 16 MP Front...,Memory Card Not Supported,Android v13


In [598]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1020 entries, 0 to 1019
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   model      1020 non-null   object 
 1   price      1020 non-null   object 
 2   rating     879 non-null    float64
 3   sim        1020 non-null   object 
 4   processor  1020 non-null   object 
 5   ram        1020 non-null   object 
 6   battery    1020 non-null   object 
 7   display    1020 non-null   object 
 8   camera     1019 non-null   object 
 9   card       1013 non-null   object 
 10  os         1003 non-null   object 
dtypes: float64(1), object(10)
memory usage: 87.8+ KB


In [599]:
print('Number of columns in the Data : ',df.shape[1])
print('Number of Rows in the Data : ', df.shape[0])

Number of columns in the Data :  11
Number of Rows in the Data :  1020


In [600]:
df.isnull()\
        .sum()\
        .reset_index()\
        .rename(columns={'index':'columns',
                        0:'Missing values'})

,columns,Missing values
0,model,0
1,price,0
2,rating,141
3,sim,0
4,processor,0
5,ram,0
6,battery,0
7,display,0
8,camera,1
9,card,7


In [601]:
# extract brand from the Name
df['brand'] = df['model'].str.split().str[0]

In [602]:
# preprocess price feature
df['price'] = df['price'].str.replace('₹','').str.replace(',','')
df['price'] = df['price'].astype(int)
df['price']

0       54999
1       19989
2       16499
3       14999
4       24999
        ...  
1015    34990
1016    14990
1017    28990
1018    19990
1019    24990
Name: price, Length: 1020, dtype: int32

In [603]:
df['rating'] = df['rating'].astype('Int32')

In [604]:
# Checking all the unique features in sim column
sim_features = pd.Series(df['sim'].str.split(',').sum())
sim_features.unique()

array(['Dual Sim', ' 3G', ' 4G', ' 5G', ' VoLTE', ' Wi-Fi', ' NFC',
       ' IR Blaster', ' Vo5G', 'Single Sim', 'Wi-Fi'], dtype=object)

In [605]:
sim_features.value_counts().to_frame()

,count
3G,1000
4G,999
Dual Sim,998
Wi-Fi,994
VoLTE,983
5G,549
NFC,393
IR Blaster,159
Single Sim,21
Vo5G,12


We can use regex for creating individual feature columns later


In [606]:
# let's check the same for processor
df['processor'].str.split(',').str[0].unique()

array(['Snapdragon 8 Gen2', 'Snapdragon 695', 'Exynos 1330',
       'Snapdragon  695', 'Dimensity 1080', 'Snapdragon  750G',
       'Bionic A15', 'Snapdragon 778G Plus', 'Dimensity  1300',
       'Dimensity 700 5G', 'Helio P35', 'Snapdragon 8+ Gen1',
       'Dimensity 8100 Max', 'Helio G70', 'Snapdragon  8+ Gen1',
       'Dimensity 1300', 'Snapdragon 695 ', 'Snapdragon 4 Gen 1',
       'Dimensity 9000 Plus', 'Snapdragon 865', 'Bionic  A16',
       'Dimensity  900', 'Snapdragon  8 Gen2', 'Snapdragon 778G',
       'Dimensity  930', 'A13 Bionic', 'Dimensity  900 5G',
       'Exynos  2100', 'Snapdragon  8 Gen1', 'Snapdragon  7 Gen1',
       ' Snapdragon  695', 'Dimensity 800U', 'Snapdragon 680',
       'Exynos  1280', 'Snapdragon  680', 'Unisoc T612',
       'Snapdragon 7 Gen1', 'Dimensity 8200', 'Snapdragon 480+',
       'Helio G99', 'Dimensity 810 5G', 'Helio G95', 'Dimensity 720',
       'Snapdragon 870', 'Dimensity  810 5G', 'Helio G35',
       'Dimensity  920 5G', 'Snapdragon  4 Gen 1

-  there phone without any proccessor name
- some of the data point have value like: 
    - 'No 3G' value.
    - '1\u2009GB RAM
    - '2000\u2009mAh Battery'
    - '32\u2009GB inbuilt'
    - '256\u2009MB RAM'
    - '8\u2009MB RAM'

In [607]:
pd.Series(df['processor']
          .str.split(',')
          .str[1:].sum())\
          .value_counts()\
          .to_frame()

,count
Octa Core,866
2 GHz Processor,146
2.2 GHz Processor,135
2.4 GHz Processor,128
3.2 GHz Processor,94
2.3 GHz Processor,86
3 GHz Processor,53
2.84 GHz Processor,36
Quad Core,33
Hexa Core,31


Same a processor , there are values here that not meant to be here

In [608]:
pd.Series(df['ram'].str.split().sum()).unique()

array(['12', 'GB', 'RAM,', '256', 'inbuilt', '6', '128', '4', '64', '8',
       '3', '32', '16', '2', '512', 'MB', '18', '1', 'TB', 'Unisoc',
       'T107', '48', 'Single', 'Core,', '208', 'MHz', 'Processor', 'T117',
       '1325', 'mAh', 'Battery', 'Dual', '500', '800', '3.6', 'inches,',
       '240', 'x', '320', 'px', 'Display', '1750', '200', '1200', '1.5',
       '120', '2.4', '640', '1136', '1000', '1470', '1800', 'No', 'Wifi'],
      dtype=object)

- in ram column we have 
    - battery data, 
    - Display 
    - wifi
    - proccessor

In [609]:
df['battery'].unique()

array(['5000\u2009mAh Battery with 100W Fast Charging',
       '5000\u2009mAh Battery with 33W Fast Charging',
       '5000\u2009mAh Battery with 15W Fast Charging',
       '5000\u2009mAh Battery with Fast Charging',
       '5000\u2009mAh Battery with 67W Fast Charging',
       '5000\u2009mAh Battery with 25W Fast Charging',
       '3279\u2009mAh Battery with Fast Charging',
       '4980\u2009mAh Battery with 120W Fast Charging',
       '4500\u2009mAh Battery with 33W Fast Charging',
       '4500\u2009mAh Battery with 80W Fast Charging',
       '5000\u2009mAh Battery with 18W Fast Charging',
       '5000\u2009mAh Battery with 45W Fast Charging',
       '3240\u2009mAh Battery with Fast Charging',
       '5000\u2009mAh Battery with 10W Fast Charging',
       '4700\u2009mAh Battery with 80W Fast Charging',
       '5000\u2009mAh Battery with 80W Fast Charging',
       '4830\u2009mAh Battery with 66W Fast Charging',
       '4800\u2009mAh Battery with 100W Fast Charging',
       '4500\u2009m

- As same, Battery column have values of :
    - display
    - ram
    - rom
    - Inches
    -  'No Rear Camera'
    - '5\u2009MP Rear & 5\u2009MP Front Camera'

In [610]:
pd.Series(df['display'].str.split(',').sum()).unique()

array(['6.7 inches', ' 1440\u2009x\u20093216\u2009px',
       ' 120 Hz Display with Punch Hole', '6.59 inches',
       ' 1080\u2009x\u20092412\u2009px', '6.6 inches',
       ' 1080\u2009x\u20092408\u2009px',
       ' 90 Hz Display with Water Drop Notch', '6.55 inches',
       ' 1080\u2009x\u20092400\u2009px',
       ' 120 Hz Display with Water Drop Notch', '6.1 inches',
       ' 1170\u2009x\u20092532\u2009px Display with Small Notch',
       '6.67 inches', '6.43 inches', ' 90 Hz Display with Punch Hole',
       '6.72 inches', '6.56 inches', ' 720\u2009x\u20091612\u2009px',
       '6.58 inches', '6.8 inches', ' 1440\u2009x\u20093088\u2009px',
       '6.51 inches',
       ' 720\u2009x\u20091600\u2009px Display with Water Drop Notch',
       ' 720\u2009x\u20091612\u2009px Display with Water Drop Notch',
       ' 1080\u2009x\u20092376\u2009px', '6.5 inches',
       ' 1290\u2009x\u20092796\u2009px', ' 120 Hz Display', '6.44 inches',
       ' 828\u2009x\u20091792\u2009px Display with Large N

messy data
- Camera
- Battery
- upto 64\u2009GB', 'iOS v12',
- 'No FM Radio'
- 'Memory Card Supported', ' upto 32\u2009GB',

In [611]:
pd.Series(df['camera'].fillna('').str.split('+').sum()).unique()

array(['50\u2009MP ', ' 48\u2009MP ',
       ' 32\u2009MP Triple Rear & 16\u2009MP Front Camera', '64\u2009MP ',
       ' 2\u2009MP ', ' 2\u2009MP Triple Rear & 16\u2009MP Front Camera',
       ' 2\u2009MP Triple Rear & 13\u2009MP Front Camera', ' 8\u2009MP ',
       '108\u2009MP ', ' 2\u2009MP Triple Rear & 8\u2009MP Front Camera',
       '12\u2009MP ', ' 12\u2009MP Dual Rear & 12\u2009MP Front Camera',
       '200\u2009MP ', ' 50\u2009MP Dual Rear & 16\u2009MP Front Camera',
       ' 2\u2009MP Triple Rear & 32\u2009MP Front Camera',
       ' 2\u2009MP Dual Rear & 16\u2009MP Front Camera',
       ' 2\u2009MP Dual Rear & 8\u2009MP Front Camera',
       '200\u2009MP Quad Rear & 12\u2009MP Front Camera', '13\u2009MP ',
       ' 2\u2009MP Dual Rear & 5\u2009MP Front Camera', ' 12\u2009MP ',
       '48\u2009MP ', ' 8\u2009MP Triple Rear & 32\u2009MP Front Camera',
       ' 12\u2009MP Triple Rear & 12\u2009MP Front Camera',
       ' 2\u2009MP Triple Rear & 50\u2009MP Front Camera', ' 13\u20

Messy data
- Display
- Memory
- FM
- Kaios
- TOF 3D
- 'Android v5.0 (Lollipop)

In [612]:
df['card'].unique()

array(['Memory Card Not Supported',
       'Memory Card (Hybrid), upto 1\u2009TB',
       'Memory Card Supported, upto 1\u2009TB', 'Android v12',
       'Memory Card Supported', 'Android v13', 'Memory Card (Hybrid)',
       'Memory Card Supported, upto 512\u2009GB',
       '64\u2009MP + 10\u2009MP + 8\u2009MP Triple Rear & 32\u2009MP Front Camera',
       '50\u2009MP Quad Rear & 16\u2009MP Front Camera',
       'Memory Card Supported, upto 256\u2009GB', 'iOS v14',
       'Memory Card Supported, upto 2\u2009TB',
       'Memory Card Supported, upto 128\u2009GB', 'Android v11',
       '5\u2009MP Rear Camera',
       '50\u2009MP + 48\u2009MP + 32\u2009MP Triple Rear & 32\u2009MP + 32\u2009MP Dual Front Camera',
       'Memory Card (Hybrid), upto 256\u2009GB',
       'Memory Card (Hybrid), upto 128\u2009GB',
       '50\u2009MP + 13\u2009MP + 8\u2009MP Triple Rear & 20\u2009MP Front Camera',
       '2\u2009MP Rear Camera', 'Memory Card (Hybrid), upto 512\u2009GB',
       '12\u2009MP + 12\u20

messy data in card colunmns
- 'Android v13'
- '64\u2009MP + 10\u2009MP + 8\u2009MP Triple Rear & 32\u2009MP Front Camera',
- 'iOS v14'
- 'Browser'

In [613]:
df['os'].unique()

array(['Android v13', 'Android v12', 'iOS v16', 'No FM Radio', 'iOS v15',
       'Bluetooth', 'Android v11', 'Android v10', 'iOS v13',
       'HarmonyOS v2', 'iOS v15.0', 'Android v10.0',
       'Android v8.1 (Oreo)', 'Memory Card Supported, upto 32\u2009GB',
       'Memory Card Not Supported', 'iOS v13.0', 'Android v11.0',
       'Android v9.0 (Pie)', 'Android v12.1', 'iOS v12.3',
       'Memory Card Supported', 'Memory Card Supported, upto 256\u2009GB',
       '0.3\u2009MP Rear Camera', 'KAI OS', 'iOS v17',
       'Android v4.4.2 (KitKat)', nan, 'Android v5.1 (Lollipop)',
       'Android v5.1.1 (Lollipop)', 'Hongmeng OS v3.0',
       'Android v8.0 (Oreo)', 'Memory Card Supported, upto 64\u2009GB',
       'Memory Card (Hybrid), upto 2\u2009TB',
       'Android v6.0 (Marshmallow)', 'Pragati OS (Powered by Android)',
       'Memory Card Supported, upto 16\u2009GB', 'Memory Card (Hybrid)',
       'RTOS (Series 30+)', 'Harmony v2.0', 'Hongmeng OS v4.0',
       'Memory Card (Hybrid), upto 

Messy data in os feature
- FM
- Bluetooth
- upto 32\u2009GB'
- 'Memory Card Not Supported'
-  Camera

In [614]:
def shift_values(row):
    if row['processor'] =='No 3G':
        row['processor'] = row['ram']
        row['ram'] = row['battery']
        row['battery'] = row['display']
        row['display'] = row['camera']
        row['camera'] = row['card']
        row['card'] = row['os']
    return row

df = df.apply(shift_values,axis=1)

In [615]:
def shift_proc_ram(row):
    if 'RAM' in row['processor']:

        row['card'] = row['camera']        
        row['camera'] = row['display']
        row['display'] = row['battery']
        row['battery'] = row['ram']
        row['ram'] = row['processor']
        row['processor'] = np.NaN
    return row
df = df.apply(shift_proc_ram,axis=True)

In [616]:
def shift_proc_battery(row):
    if 'Battery' in row['processor']:

        row['os'] = row['camera']        
        row['card'] = row['display']
        row['camera'] = row['battery']
        row['display'] = row['ram']
        row['battery'] = row['processor']
        row['processor'] = np.NaN
        row['ram'] = np.NaN
    return row

df['processor'].fillna('NA',inplace=True)
df = df.apply(shift_proc_battery,axis=1)
df['processor'].fillna('NA',inplace=True)


In [617]:
df[df['processor'].str.contains('inbuilt',case=True)].index
df.drop(index=754,inplace=True)

In [618]:
df['dual_sim'] = df['sim'].str.contains(pat='Dual',case=False)
df['is_5G'] = df['sim'].str.contains(pat='5G',case=False)
df['is_NFC'] = df['sim'].str.contains('NFC',case=False)
df['is_VoLTE'] = df['sim'].str.contains('volte',case=False)
df['is_IRblaster'] = df['sim'].str.contains('IR Blaster	',case=False)
df['is_Wifi'] = df['sim'].str.contains('wi[\S]+fi',case=False,regex=True)

In [619]:
df['processor_name'] = df['processor'].str.split(',').str[0]
df['processor_ghz'] = df['processor'].str.extract('([0-9\W]+.GHz)',flags=re.IGNORECASE)
df['processor_core'] = df['processor'].str.extract(pat='(\w+ Core)',flags=re.IGNORECASE)

In [620]:
df['RAM_GB'] = df['ram'].str.extract('([\s A-Za-z0-9]+ RAM)')
df['ROM'] = df['ram'].str.extract(r'(\d+\sGB inbuilt)')

In [621]:
df['battery_mah'] = df['battery'].str.extract('(\d+.mAh)')
df['fastcharging_W'] = df['battery'].str.extract('([\d.]+W)')
df['is_fastcharging'] = df['battery'].str.contains('fast',case=False)

In [622]:
def pat_extraction(pat,column):
   return df[column].str.extract(pat)

In [623]:
df['diplay_inches'] = pat_extraction('([\d.]+ inches)','display')
df['display_resolution'] = pat_extraction('([0-9x\s\W]+px)','display')
df['display_refreshrate'] = pat_extraction('(\d+ Hz)','display')
df['display_notch'] = df['display'].str.extract('(with .+)')

In [624]:
df['main_camera'] = df['camera'].str.split(' ').str[0].str.extract('(\d.+MP)')
df['secondary_camera'] = df['camera'].str.split('+').str[1].str.extract('(\d.+MP)',expand=False)
df['third_camera'] = df['camera'].str.split(' ').str[4].str.extract('(\d.+MP)',expand=False)
df['num_cameras'] = df['camera'].str.extract('(\w+ Rear)',expand=False)
df['front_camera'] = df['camera'].str.extract('([\dA-Z\W]+) Fr')

In [625]:
df['is_card_supported'] = df['card'].str.contains('not supported',case=False).eq(False)
df['card_upto'] = df['card'].str.extract('(upto.+)')
df['is_card_hybrid']= df['card'].str.contains('hybrid',case=False)

In [626]:
df['os_type'] = df['os'].str.split(' ').str.get(0)
df['os_version'] = df['os'].str.extract('(v[\d.]+)')

In [627]:
#leaning Messy data

In [628]:
df.drop(df[df['processor'].str.contains('Wifi')].index,inplace=True)
df.drop(df[df['brand']=='Nokia'].index,inplace=True)
df.drop(columns=['processor'],inplace=True)
df.drop('sim',axis=1,inplace=True)

In [633]:
df.loc[df['battery'].str.contains('display',case=False)].head()

,model,price,rating,ram,battery,display,camera,card,os,brand,dual_sim,is_5G,is_NFC,is_VoLTE,is_IRblaster,is_Wifi,processor_name,processor_ghz,processor_core,RAM_GB,ROM,battery_mah,fastcharging_W,is_fastcharging,diplay_inches,display_resolution,display_refreshrate,display_notch,main_camera,secondary_camera,third_camera,num_cameras,front_camera,is_card_supported,card_upto,is_card_hybrid,os_type,os_version
111,Apple iPhone 12,51999,74,"4 GB RAM, 64 GB inbuilt","6.1 inches, 1170 x 2532 px Display with Large ...",12 MP + 12 MP Dual Rear & 12 MP Front Camera,Memory Card Not Supported,iOS v14,No FM Radio,Apple,True,True,True,True,False,True,Bionic A14,", 3.1 GHz",Hexa Core,4 GB RAM,64 GB inbuilt,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,False,NaN,NaN
149,Apple iPhone 12 Mini,40999,74,"4 GB RAM, 64 GB inbuilt","5.4 inches, 1080 x 2340 px Display",12 MP + 12 MP Dual Rear & 12 MP Front Camera,Memory Card Not Supported,iOS v14,No FM Radio,Apple,True,True,True,True,False,True,Bionic A14,", 3.1 GHz",Hexa Core,4 GB RAM,64 GB inbuilt,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,False,NaN,NaN
307,Apple iPhone 12 (128GB),55999,75,"4 GB RAM, 128 GB inbuilt","6.1 inches, 1170 x 2532 px Display with Large ...",12 MP + 12 MP Dual Rear & 12 MP Front Camera,Memory Card Not Supported,iOS v14,No FM Radio,Apple,True,True,True,True,False,True,Bionic A14,", 3.1 GHz",Hexa Core,4 GB RAM,128 GB inbuilt,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,False,NaN,NaN
363,Apple iPhone 12 Mini (128GB),45999,75,"4 GB RAM, 128 GB inbuilt","5.4 inches, 1080 x 2340 px Display",12 MP + 12 MP Dual Rear & 12 MP Front Camera,Memory Card Not Supported,iOS v14,No FM Radio,Apple,True,True,True,True,False,True,Bionic A14,", 3.1 GHz",Hexa Core,4 GB RAM,128 GB inbuilt,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,False,NaN,NaN
439,Apple iPhone SE 3 2022,43900,<NA>,64 GB inbuilt,"4.7 inches, 750 x 1334 px Display",12 MP Rear & 7 MP Front Camera,Memory Card Not Supported,iOS v15,No FM Radio,Apple,True,True,True,True,False,True,Bionic A15,", 3.22 GHz",Hexa Core,NaN,64 GB inbuilt,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,False,NaN,NaN


In [631]:
df.loc[df['battery'].str.contains('display',case=False),['os_type','os_version']] = df.loc[df['battery'].str.contains('display',case=False)]['card'].str.split(expand=True)

In [ ]:
df.loc[df['battery'].str.contains('display',case=False),['os_type']] = df.loc[df['battery'].str.contains('display',case=False)]['card'].str.split(expand=True)[0]
df.loc[df['battery'].str.contains('display',case=False),['os_version']] = df.loc[df['battery'].str.contains('display',case=False)]['card'].str.split(expand=True)[1]

In [594]:
df.loc[df['battery'].str.contains('display',case=False),['os_version','os_type']]

,os_version,os_type
111,v14,iOS
149,v14,iOS
307,v14,iOS
363,v14,iOS
439,v15,iOS
448,v15,iOS
628,v14.0,iOS
762,v16,iOS
853,v14.0,iOS
913,v14,iOS


In [ ]:
df.drop(columns='battery',inplace=True)
df.drop(columns='ram',inplace=True)

In [655]:
df[df['display'].str.contains('camera',case=False)]

,model,price,rating,display,camera,card,os,brand,dual_sim,is_5G,is_NFC,is_VoLTE,is_IRblaster,is_Wifi,processor_name,processor_ghz,processor_core,RAM_GB,ROM,battery_mah,fastcharging_W,is_fastcharging,diplay_inches,display_resolution,display_refreshrate,display_notch,main_camera,secondary_camera,third_camera,num_cameras,front_camera,is_card_supported,card_upto,is_card_hybrid,os_type,os_version
111,Apple iPhone 12,51999,74,12 MP + 12 MP Dual Rear & 12 MP Front Camera,Memory Card Not Supported,iOS v14,No FM Radio,Apple,True,True,True,True,False,True,Bionic A14,", 3.1 GHz",Hexa Core,4 GB RAM,64 GB inbuilt,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,False,NaN,NaN
149,Apple iPhone 12 Mini,40999,74,12 MP + 12 MP Dual Rear & 12 MP Front Camera,Memory Card Not Supported,iOS v14,No FM Radio,Apple,True,True,True,True,False,True,Bionic A14,", 3.1 GHz",Hexa Core,4 GB RAM,64 GB inbuilt,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,False,NaN,NaN
307,Apple iPhone 12 (128GB),55999,75,12 MP + 12 MP Dual Rear & 12 MP Front Camera,Memory Card Not Supported,iOS v14,No FM Radio,Apple,True,True,True,True,False,True,Bionic A14,", 3.1 GHz",Hexa Core,4 GB RAM,128 GB inbuilt,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,False,NaN,NaN
363,Apple iPhone 12 Mini (128GB),45999,75,12 MP + 12 MP Dual Rear & 12 MP Front Camera,Memory Card Not Supported,iOS v14,No FM Radio,Apple,True,True,True,True,False,True,Bionic A14,", 3.1 GHz",Hexa Core,4 GB RAM,128 GB inbuilt,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,False,NaN,NaN
439,Apple iPhone SE 3 2022,43900,<NA>,12 MP Rear & 7 MP Front Camera,Memory Card Not Supported,iOS v15,No FM Radio,Apple,True,True,True,True,False,True,Bionic A15,", 3.22 GHz",Hexa Core,NaN,64 GB inbuilt,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,False,NaN,NaN
448,Apple iPhone 15 Pro,130990,75,50 MP + 12 MP + 12 MP Triple Rear & 12 MP Fron...,Memory Card Not Supported,iOS v15,No FM Radio,Apple,True,True,True,True,False,True,Bionic A16,NaN,NaN,8 GB RAM,128 GB inbuilt,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,False,NaN,NaN
628,Apple iPhone 12 Pro (512GB),139900,80,12 MP + 12 MP + 12 MP Triple Rear & 12 MP Fron...,Memory Card Not Supported,iOS v14.0,No FM Radio,Apple,True,True,True,True,False,True,Bionic A14,", 3.1 GHz",Hexa Core,6 GB RAM,512 GB inbuilt,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,False,NaN,NaN
762,Apple iPhone SE 4,49990,60,12 MP Rear & 10.8 MP Front Camera,Memory Card Not Supported,iOS v16,No FM Radio,Apple,True,True,True,True,False,True,Bionic A15,", 3.22 GHz",Hexa Core,NaN,64 GB inbuilt,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,False,NaN,NaN
853,Apple iPhone 12 Pro (256GB),119900,80,12 MP + 12 MP + 12 MP Triple Rear & 12 MP Fron...,Memory Card Not Supported,iOS v14.0,No FM Radio,Apple,True,True,True,True,False,True,Bionic A14,", 3.1 GHz",Hexa Core,6 GB RAM,256 GB inbuilt,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,False,NaN,NaN
913,Apple iPhone 12 Mini (256GB),55999,75,12 MP + 12 MP Dual Rear & 12 MP Front Camera,Memory Card Not Supported,iOS v14,No FM Radio,Apple,True,True,True,True,False,True,Bionic A14,", 3.1 GHz",Hexa Core,4 GB RAM,256 GB inbuilt,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,False,NaN,NaN


In [ ]:
def shift_proc_ram(row):
    if 'Camera' in row['display']:

        row['os'] = row['card']
        row['card'] = row['camera']        
        row['camera'] = row['display']
        
    
    return row
df = 

In [656]:
df.apply(shift_proc_ram,axis=True)

KeyError: 'processor'